In [1]:
import gensim
import pandas as pd

In [75]:
df = pd.read_csv('data/judicial_orders.csv')
code = pd.read_csv('data/criminal_code.csv')

In [6]:
df['lemmas'] = df['lemmas'].map(eval)
df['labels'] = df['labels'].map(eval)

In [7]:
td = lambda row: gensim.models.doc2vec.TaggedDocument(words=row['lemmas'], tags=[row['labels'][0]])
documents = df.apply(td, axis=1).tolist()

In [19]:
model = gensim.models.doc2vec.Doc2Vec(alpha=0.025, min_alpha=0.025)  # use fixed learning rate
model.build_vocab(documents)

model.train(documents, total_examples=model.corpus_count, epochs=10, 
            start_alpha=0.025, end_alpha=0.02)

In [77]:
doc_vectors = dict(zip(model.docvecs.offset2doctag, model.docvecs.vectors_docs))

In [92]:
code['doc2vec'] = code['number'].map(lambda x: list(doc_vectors[x]) if x in doc_vectors else None)

In [96]:
code.dropna(subset=['doc2vec']).rename(columns={'doc2vec':'vectors'}).to_csv('results/doc2vec.csv', index=False)